In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [9]:
df = pd.read_csv("/content/drive/MyDrive/Fake News using LSTM/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df = df.dropna()

In [11]:
# Get Independent Features:
X = df.drop('label', axis = 1)

In [12]:
# Get Dependent Features:
y = df['label']

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
# Vocabulary Size:
voc_size = 5000

## One hot Encoding:

In [19]:
messages = X.copy()

In [20]:
messages.reset_index(inplace = True)

In [23]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
## Data Preprocessing:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []

for i in range(0,len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[38, 2888, 4686, 4599, 3915, 1255, 4061, 867, 4459, 3668],
 [3896, 223, 2376, 1024, 3623, 1384, 1620],
 [2726, 4062, 4721, 4136],
 [2845, 3097, 798, 128, 2311, 2021],
 [4187, 3623, 2207, 4059, 1293, 1414, 3623, 3124, 3159, 1331],
 [1853,
  2408,
  3744,
  3484,
  1854,
  3075,
  3860,
  2348,
  3608,
  2032,
  4141,
  4048,
  3934,
  1654,
  1620],
 [2244, 2702, 4694, 4667, 1776, 536, 2456, 113, 203, 4052, 2416],
 [2600, 204, 4508, 4147, 4169, 237, 3075, 2427, 203, 4052, 2416],
 [1530, 447, 611, 3975, 2270, 3848, 1353, 2235, 3075, 1073],
 [215, 430, 1571, 4618, 1595, 1473, 3594, 2874],
 [1839, 4802, 335, 343, 4441, 3135, 4380, 2504, 1652, 344, 661],
 [128, 4772, 3915, 3848, 3075, 4169],
 [4272, 3240, 3048, 2729, 548, 4122, 3153, 1902, 1653],
 [1889, 951, 3339, 1530, 1500, 2810, 235, 203, 4052, 2416],
 [3822, 1892, 1988, 715, 243, 203, 4052, 2416],
 [72, 2449, 4221, 129, 4875, 986, 2330, 2944, 331, 4865],
 [3107, 1750, 223],
 [715, 4369, 4436, 3354, 3075, 4140, 1757, 1620],
 [2797, 764

## Embedding Representation:

In [29]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ...  867 4459 3668]
 [   0    0    0 ... 3623 1384 1620]
 [   0    0    0 ... 4062 4721 4136]
 ...
 [   0    0    0 ...  203 4052 2416]
 [   0    0    0 ... 3253 4149  583]
 [   0    0    0 ... 1695 1281 2423]]


In [30]:
len(embedded_docs)

18285

## Model Creation:

In [32]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model.add(LSTM(100)) # 100 Neurons
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

In [36]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [38]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state=42)

## Model Training:

In [66]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 11s 45ms/step - loss: 0.3362 - accuracy: 0.8387 - val_loss: 0.2002 - val_accuracy: 0.9155
Epoch 2/10
192/192 [==============================] - 8s 42ms/step - loss: 0.1455 - accuracy: 0.9409 - val_loss: 0.2015 - val_accuracy: 0.9165
Epoch 3/10
192/192 [==============================] - 8s 42ms/step - loss: 0.1054 - accuracy: 0.9595 - val_loss: 0.2342 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0851 - accuracy: 0.9696 - val_loss: 0.2399 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0652 - accuracy: 0.9761 - val_loss: 0.2729 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0547 - accuracy: 0.9802 - val_loss: 0.3481 - val_accuracy: 0.9080
Epoch 7/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0413 - accuracy: 0.9864 - val_loss: 0.3785 - val_accuracy: 0.900

## Hyper Parameter Tuning:
Adding dropout layer.

In [61]:
from tensorflow.keras.layers import Dropout
## Creating model:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100)) # 100 Neurons
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

## Performance Metrics and Accuracy:

In [67]:
y_pred = (model.predict(X_test) > 0.5).astype('int32')

In [68]:
from sklearn.metrics import confusion_matrix

In [69]:
confusion_matrix(y_test, y_pred)
# False Positive and False Negative is really less as compared to True Positive and True Negative

array([[3093,  326],
       [ 239, 2377]])

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9063794531897266